In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[*]").getOrCreate()

23/06/18 14:06:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [22]:
book_df = spark.read.csv('gs://process-data-bucket/output/book_data.csv',header=True,inferSchema=True)
transaction_df = spark.read.csv('gs://process-data-bucket/output/transaction_data.csv',header=True,inferSchema=True)
conversion_df = spark.read.csv('gs://process-data-bucket/output/conversion_rate_data.csv',header=True,inferSchema=True)

In [3]:
#change col name to remove after join
book_df = book_df.withColumnRenamed('Book_ID','join_temp')

df = transaction_df.join(book_df,transaction_df.book_id==book_df.join_temp,"left")
df = df.drop('join_temp')
df = df.withColumn('date',to_date('timestamp'))
df.show()

+-------------------+--------+-------+--------------------+--------------------+--------------------+------------------+-------------------+------------------+--------------------+--------------------+------+--------------------+-------+----------+
|          timestamp| user_id|book_id|             country|          Book Title|       Book Subtitle|       Book Author|      Book Narrator|     Audio Runtime|      Audiobook_Type|          Categories|Rating|Total No. of Ratings|  Price|      date|
+-------------------+--------+-------+--------------------+--------------------+--------------------+------------------+-------------------+------------------+--------------------+--------------------+------+--------------------+-------+----------+
|2021-05-01 00:00:01|ad8eca41|   1584|            Portugal|        Sycamore Row|                null|      John Grisham|       Michael Beck|20 hrs and 46 mins|Unabridged Audiobook|Mystery, Thriller...|   4.6|             25863.0|$35.00 |2021-05-01|
|202

In [4]:
temp = conversion_df
temp = temp.withColumn('date',to_date('date'))
temp = temp.withColumnRenamed('date','join_temp')
temp.show()

+----------+---------------+
| join_temp|conversion_rate|
+----------+---------------+
|2021-04-01|         31.194|
|2021-04-02|          31.29|
|2021-04-03|         31.256|
|2021-04-04|         31.244|
|2021-04-05|         31.342|
|2021-04-06|         31.288|
|2021-04-07|         31.387|
|2021-04-08|         31.433|
|2021-04-09|         31.451|
|2021-04-10|         31.454|
|2021-04-11|         31.462|
|2021-04-12|         31.482|
|2021-04-13|         31.496|
|2021-04-14|         31.329|
|2021-04-15|         31.165|
|2021-04-16|         31.203|
|2021-04-17|         31.204|
|2021-04-18|         31.226|
|2021-04-19|         31.203|
|2021-04-20|         31.281|
+----------+---------------+
only showing top 20 rows



In [5]:
df = df.join(temp,df.date==temp.join_temp,"left")
df = df.drop('join_temp')
df.show()

+-------------------+--------+-------+--------------------+--------------------+--------------------+------------------+-------------------+------------------+--------------------+--------------------+------+--------------------+-------+----------+---------------+
|          timestamp| user_id|book_id|             country|          Book Title|       Book Subtitle|       Book Author|      Book Narrator|     Audio Runtime|      Audiobook_Type|          Categories|Rating|Total No. of Ratings|  Price|      date|conversion_rate|
+-------------------+--------+-------+--------------------+--------------------+--------------------+------------------+-------------------+------------------+--------------------+--------------------+------+--------------------+-------+----------+---------------+
|2021-05-01 00:00:01|ad8eca41|   1584|            Portugal|        Sycamore Row|                null|      John Grisham|       Michael Beck|20 hrs and 46 mins|Unabridged Audiobook|Mystery, Thriller...|   4

In [7]:
# remove $ and change data type to float
df = df.withColumn('Price', regexp_replace('Price', '^\W{1}', ''))
df = df.withColumn('Price',df.Price.cast('float'))
df.show()

+-------------------+--------+-------+--------------------+--------------------+--------------------+------------------+-------------------+------------------+--------------------+--------------------+------+--------------------+-----+----------+---------------+
|          timestamp| user_id|book_id|             country|          Book Title|       Book Subtitle|       Book Author|      Book Narrator|     Audio Runtime|      Audiobook_Type|          Categories|Rating|Total No. of Ratings|Price|      date|conversion_rate|
+-------------------+--------+-------+--------------------+--------------------+--------------------+------------------+-------------------+------------------+--------------------+--------------------+------+--------------------+-----+----------+---------------+
|2021-05-01 00:00:01|ad8eca41|   1584|            Portugal|        Sycamore Row|                null|      John Grisham|       Michael Beck|20 hrs and 46 mins|Unabridged Audiobook|Mystery, Thriller...|   4.6|   

In [13]:
# Create THB column
df.withColumn('THBPrice',df.Price*df.conversion_rate).show()
# drop date column
df = df.drop('date')

In [30]:
df.write.csv('gs://process-data-bucket/output/audible_data.csv', header = True)

In [14]:
df.dtypes

[('timestamp', 'timestamp'),
 ('user_id', 'string'),
 ('book_id', 'int'),
 ('country', 'string'),
 ('Book Title', 'string'),
 ('Book Subtitle', 'string'),
 ('Book Author', 'string'),
 ('Book Narrator', 'string'),
 ('Audio Runtime', 'string'),
 ('Audiobook_Type', 'string'),
 ('Categories', 'string'),
 ('Rating', 'string'),
 ('Total No. of Ratings', 'double'),
 ('Price', 'float'),
 ('conversion_rate', 'double')]